In [1]:
!pip install transformers
!pip install simpletransformers

     |████████████████████████████████| 2.3MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 56.4MB/s 
     |████████████████████████████████| 901kB 62.5MB/s 
     |████████████████████████████████| 225kB 6.8MB/s 
     |████████████████████████████████| 235kB 10.5MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 8.2MB 11.8MB/s 
     |████████████████████████████████| 1.2MB 66.4MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 1.8MB 52.0MB/s 
     |████████████████████████████████| 122kB 57.8MB/s 
     |████████████████████████████████| 112kB 59.4MB/s 
     |████████████████████████████████| 245kB 48.2MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 174kB 65.7MB/s 
     |████████████████████████████████| 4.2MB 43.2MB/s 
     |████████████████████████████████| 112kB 71.4MB/s 
     |████████████████████████████████| 102kB 15.2MB/s 

In [52]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import sklearn
# import ipdb


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer

from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig
import logging

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive


In [10]:
documents_train = pd.read_csv("/content/drive/MyDrive/round1/data_round_1/documents_en_train.csv")
sentences_train = pd.read_csv("/content/drive/MyDrive/round1/data_round_1/sentences_en_train.csv")

documents_val = pd.read_csv("/content/drive/MyDrive/round1/data_round_1/documents_en_val.csv")
sentences_val = pd.read_csv("/content/drive/MyDrive/round1/data_round_1/sentences_en_val.csv")

documents_test = pd.read_csv("/content/drive/MyDrive/round1/data_round_1/documents_en_test.csv")
sentences_test = pd.read_csv("/content/drive/MyDrive/round1/data_round_1/sentences_en_test.csv")

In [ ]:
documents_train = pd.read_csv("data_round_1/documents_en_train.csv")
sentences_train = pd.read_csv("data_round_1/sentences_en_train.csv")

documents_val = pd.read_csv("data_round_1/documents_en_val.csv")
sentences_val = pd.read_csv("data_round_1/sentences_en_val.csv")

documents_test = pd.read_csv("data_round_1/documents_en_test.csv")
sentences_test = pd.read_csv("data_round_1/sentences_en_test.csv")
#immap_sector_name_to_id.json

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [12]:
device

device(type='cuda')

## Investigate Document data

In [13]:
documents_train.head()

,project_name,country_code,lang_code,doc_id,doc_text,doc_url
0,IMMAP/DFS Syria,SYR,en,48582,This website uses cookies to improve your expe...,https://www.syriahr.com/en/203844/
1,IMMAP/DFS Syria,SYR,en,41032,Please enable Cookies and reload the page.\n\n...,https://www.syriahr.com/en/187230/
2,IMMAP/DFS Burkina Faso,BFA,en,41306,When Burkina Faso’s government called for civi...,https://www.thenewhumanitarian.org/news-featur...
3,IMMAP/DFS Syria,SYR,en,43779,DAMASCUS – The United Nations World Food Progr...,https://reliefweb.int/report/syrian-arab-repub...
4,IMMAP/DFS Nigeria,NGA,en,42212,Overview The continuation of conflict in North...,https://reliefweb.int/sites/reliefweb.int/file...


## Investigate full text data

In [ ]:
sentences_train.head()

In [ ]:
counts = sentences_train[["sentence_id", "doc_id"]].groupby("doc_id").count()
print("Median sentence lenght of document:", np.median(counts))
counts

In [ ]:
import matplotlib.pyplot as plt
counts = sentences_train.groupby("doc_id").count()
x = counts["sentence_id"]
plt.hist(x)
plt.show()

In [ ]:
all_sectors = []
string_list = ["[", "]", ",", " "]
for topics in sentences_train["sector_ids"]:
    for topic in topics:
        if topic not in string_list:
            all_sectors.append(topic)        

In [ ]:
plt.hist(sorted(all_sectors))
plt.show()

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
sentences_train['sentence_text']

In [ ]:
train_df = pd.DataFrame({
    'text': sentences_train['sentence_text'],
    'label': sentences_train["is_relevant"]
})

In [ ]:
eval_df = pd.DataFrame({
    'text': sentences_val['sentence_text'],
    'label': sentences_val["is_relevant"],
})

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

In [ ]:
# Create a TransformerModel
model = ClassificationModel('distilbert', 'distilbert-base-uncased', use_cuda=False)

In [ ]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
result

In [ ]:
predictions, raw_outputs = model.predict(sentences_test['sentence_text'].astype(str).values.tolist())

In [ ]:
len(predictions)

In [ ]:
submit_df = pd.DataFrame({
    'doc_id': sentences_test['doc_id'],
    'sentence_id': sentences_test["sentence_id"],
    'is_relevant': predictions,
})

In [ ]:
submit_df.to_csv("submissions/submission_relevant_baseline", index=False)

### Multi-Label Classification

In [14]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [15]:
sentences_train = sentences_train.loc[sentences_train["is_relevant"] == 1]
sentences_val = sentences_val.loc[sentences_val["is_relevant"] == 1]

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [16]:
def process_sector_ids(sentences):
    mlb = MultiLabelBinarizer()
    sectors = pd.DataFrame(mlb.fit_transform(sentences["sector_ids"]),columns=mlb.classes_)
    sectors = sectors.drop([",", "[", "]", " "], axis = 1)
    sectors["-1"] = 0
    sectors["-1"][sectors.sum(axis=1) == 0] = 1

    return sectors.values

In [17]:
train_labels = process_sector_ids(sentences_train)
val_labels = process_sector_ids(sentences_val)

In [18]:
train_labels = pd.DataFrame(train_labels)
val_labels = pd.DataFrame(val_labels)

In [19]:
sentences_train = sentences_train.reset_index()
sentences_val = sentences_val.reset_index()

In [20]:
sentences_train['labels'] = train_labels.apply(lambda x: [np.array(x)], axis=1).apply(lambda x: x[0])
sentences_val['labels'] = val_labels.apply(lambda x: [np.array(x)], axis=1).apply(lambda x: x[0])

In [21]:
sentences_train

,index,doc_id,sentence_id,sentence_text,is_relevant,sector_ids,labels
0,14,51657,14,One in five children in the country has no acc...,1,"[2, 5]","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
1,15,51657,15,Many orphaned youth live in poverty and have l...,1,"[2, 5]","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
2,16,51657,16,Some children remain living with a single pare...,1,"[2, 5]","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
3,17,51657,17,Other youth live in shelters or on the streets.,1,"[2, 5]","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
4,19,51542,0,WFP Colombia Country Brief December 2020 Opera...,1,[3],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...
41890,261836,34549,46,Anticipatory action is critical to safeguard t...,1,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
41891,261841,34549,51,Cash+ in the form of cash transfers and provis...,1,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
41892,261842,34549,52,The focus of this intervention is to support v...,1,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
41893,261843,34549,53,Cash+ in the form of cash transfers and provis...,1,[8],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


In [22]:
sentences_train = sentences_train[['sentence_text', 'labels']].copy()
sentences_val = sentences_val[['sentence_text', 'labels']].copy()

In [23]:
sentences_train.columns = ["text", "labels"]
sentences_val.columns = ["text", "labels"]

In [24]:
model_args = MultiLabelClassificationArgs(num_train_epochs=1)

In [29]:
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "distilbert", "distilbert-base-uncased", num_labels=11, use_cuda=True
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.wei

In [26]:
sentences_train

,text,labels
0,One in five children in the country has no acc...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
1,Many orphaned youth live in poverty and have l...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
2,Some children remain living with a single pare...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
3,Other youth live in shelters or on the streets.,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
4,WFP Colombia Country Brief December 2020 Opera...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
41890,Anticipatory action is critical to safeguard t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
41891,Cash+ in the form of cash transfers and provis...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
41892,The focus of this intervention is to support v...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
41893,Cash+ in the form of cash transfers and provis...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


In [48]:
# Train the model
model.train_model(sentences_train, overwrite_output_dir=True)

ValueError: ignored

In [53]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    sentences_val
)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/4927 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_0_2


Running Evaluation:   0%|          | 0/616 [00:00<?, ?it/s]

ValueError: ignored

In [55]:
model_outputs

array([[0.32617188, 0.62451172, 0.01173401, ..., 0.01495361, 0.01119995,
        0.16149902],
       [0.01390076, 0.23974609, 0.00602722, ..., 0.04434204, 0.00575256,
        0.38061523],
       [0.33886719, 0.68457031, 0.01422882, ..., 0.0241394 , 0.00930023,
        0.12536621],
       ...,
       [0.0109024 , 0.11279297, 0.00305176, ..., 0.00547028, 0.00540543,
        0.76806641],
       [0.01646423, 0.20153809, 0.00577545, ..., 0.01473236, 0.00967407,
        0.54003906],
       [0.01771545, 0.17407227, 0.00871277, ..., 0.01150513, 0.01020813,
        0.53808594]])

In [37]:
# Make predictions with the model
predictions, raw_outputs = model.predict(sentences_test['sentence_text'].astype(str).values.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/56689 [00:00<?, ?it/s]

  0%|          | 0/7087 [00:00<?, ?it/s]

In [41]:
raw_outputs

array([[0.03372192, 0.11877441, 0.00933838, ..., 0.04486084, 0.06890869,
        0.29931641],
       [0.01238251, 0.19555664, 0.01893616, ..., 0.07275391, 0.00769424,
        0.39526367],
       [0.03662109, 0.31469727, 0.00526047, ..., 0.02310181, 0.00857544,
        0.32641602],
       ...,
       [0.13464355, 0.0333252 , 0.00413513, ..., 0.00847626, 0.00307465,
        0.09173584],
       [0.06781006, 0.02722168, 0.00333786, ..., 0.00547028, 0.00222588,
        0.13745117],
       [0.00847626, 0.02207947, 0.01168823, ..., 0.01908875, 0.00503922,
        0.64257812]])

In [43]:
def hamming_score(y_true, y_pred):
    # https://link.springer.com/chapter/10.1007/978-3-540-24775-3_5
    # please check the source: https://github.com/varunshenoy/simple-metrics/blob/master/simple-metrics.py
    acc_list = []
    for yt, yp in zip(y_true, y_pred):
        set_true = set(yt)
        set_pred = set(yp)
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true & set_pred) / len(set_true | set_pred)
        acc_list.append(tmp_a)
    return np.mean(acc_list)